# Import Data

In [1]:
import pandas as pd
import numpy as np
import nltk as nk
import re 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

ModuleNotFoundError: No module named 'nltk'

In [ ]:
from itertools import chain

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
df = pd.read_excel("Data-FakeRealCOVID.xlsx")
#df = pd.read_excel("/content/drive/MyDrive/PROJET_IA/Data-FakeRealCOVID.xlsx")

In [ ]:
df

# Data cleaning

**Harmonization**


In [ ]:
def lowerTweet(tweet):
    return tweet.lower()

In [ ]:
df.tweet.loc[0]

In [ ]:
lowerTweet(df.tweet.loc[0])

**Remove URLS**

In [ ]:
# In order to remove any URL within a string in Python, you can use this RegEx function :
def removeURLs(tweet):
        return re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", tweet)


In [ ]:
df.loc[4,'tweet']

In [ ]:
removeURLs(df.loc[4,'tweet'])

**Remove emojis**

In [ ]:
#i took an example to work on 
example = df.loc[89,'tweet']
example

In [ ]:
# Remove all traces of emoji from a text file.

'''def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)
remove_emoji(df.loc[6416,'tweet'])'''

In [ ]:
#the easiet solution we found is to turn text incode into ascii wich does not include emojis and ignore all other characters that are not defined on ascii encode
#we use encode function which turns string into ascii code and ignoring none ascii characters
#then decode to make the ascii code a string again
def toAscii(tweet):
  return  (tweet.encode('ascii',errors='ignore')).decode('ascii') 


In [ ]:
toAscii(example)

**Remove punctuation & special characters**

In [ ]:
#removing all numbers ,ponctuation & special characters such as @ and #
#refrence to https://www.kerryr.net/pioneers/ascii3.htm

def removeSpecialChar(text):
    text = list(text)
    
    #remove special character & ponctuations & numbers
    listOfSpecialChars =  [*range(33,48),*range(58,65),*range(91,97),*range(123,127)]
    listOfSpecialChars.remove(39) #dont include ' in the list
    
    text = [char for char in text if (ord(char) not in listOfSpecialChars and not char.isdigit())] 
    text = ''.join(text)
    
    return text

In [ ]:
#lets test on this example
df.loc[264].tweet

In [ ]:
removeSpecialChar(df.loc[264,'tweet'])

**remove apostrophe**

In [ ]:
def removeApos(text):
    text = text.replace("'",'')
    return text

In [ ]:
#lets test on this example
df.loc[264].tweet

In [ ]:
removeApos(df.loc[264,'tweet'])

# Dimensionality Reduction




**Remove one letter words**

In [ ]:
def removeLetter(text):
    text = text.split()
    text = [word for word in text if  len(word) > 2]
    text = ' '.join(text)
    return text

**Remove stopWords**

In [ ]:
nk.download('stopwords')

from nltk.corpus import stopwords

In [ ]:
stopwords.words("english")

In [ ]:
def removeStopWords(text):
    sw = stopwords.words("english")
    text = text.split()
    text = [word for word in text if word not in sw]
    return ' '.join(text)



In [ ]:
df.loc[51].tweet

In [ ]:
removeStopWords(df.loc[51].tweet)

**Stemming**

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
    text = text.split()
    text= [ps.stem(word) for word in text]
    return ' '.join(text)

In [ ]:
removeSpecialChar(df.loc[0,'tweet']) 

In [ ]:
stem(removeSpecialChar(df.loc[0,'tweet']) )

# Data Preprocessing

In [ ]:
def preprocess(text):
    text = lowerTweet(text)
    text = removeURLs(text)
    text = toAscii(text)
    text = removeSpecialChar(text)
    text = removeStopWords(text)
    text = removeApos(text)
    text = removeStopWords(text)
    text = removeLetter(text)
    text = stem(text)
    return text

In [ ]:
df.loc[209,'tweet']

In [ ]:
preprocess(df.loc[209,'tweet'])

# Tokenization (Not Necessary)

**Tokenize**

In [ ]:
nk.download('punkt')

In [ ]:
def tokenize(text):
    return nk.word_tokenize(text)

In [ ]:
df.loc[51,'tweet']

In [ ]:
tokenize(preprocess(df.loc[51,'tweet']))

In [ ]:
tokenize('i ll see what would this give me')

**Bag of Words**

In [ ]:
def bagOfWords(df):
    tokens = []
    bag = {}
    for tweet in df.tweet:
        tokens += tokenize(preprocess(tweet))
    for token in tokens:
        if token not in bag.keys():
            bag[token] = 1
        else:
            bag[token] += 1
    return bag

In [ ]:
bagOfWords(df)

#  Weighting based on TF-IDF ( TfidfVectorizer ) 



In [ ]:
#https://towardsdatascience.com/tf-idf-for-document-ranking-from-scratch-in-python-on-real-world-dataset-796d339a4089
corpus = []
for  tweet in df.tweet :
  text = preprocess(tweet)
  corpus.append(text)



In [ ]:
corpus



**Creating TF-IDF(Term Frequency and Inverse Document Frequency)**



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf=TfidfVectorizer().fit(corpus)


**Extracting all tokens i.e Features**

In [ ]:
count_tokens=tfidf.get_feature_names()
count_tokens


**Transforming the corpus into Vector**

In [ ]:
article_vect = tfidf.transform(corpus)
article_vect.toarray()

**Displaying the final Converted Vector with Features**

In [ ]:
df_tfidf_vect=pd.DataFrame(data=article_vect.toarray(),columns=count_tokens)
df_tfidf_vect

In [ ]:
def TfIdfVec(df):
    corpus = []
    for  tweet in df.tweet :
      text = preprocess(tweet)
      corpus.append(text)

    tfidf = TfidfVectorizer().fit(corpus)

    tfidf_tokens = tfidf.get_feature_names()
    article_vect = tfidf.transform(corpus)

    df_tfidf_vect = pd.DataFrame(data=article_vect.toarray(),columns=tfidf_tokens)

    return df_tfidf_vect

# Weighting based on TF ( CountVectorizer )

In [ ]:
#https://medium.com/@cmukesh8688/tf-idf-vectorizer-scikit-learn-dbc0244a911a
corpus = []
for  tweet in df.tweet :
  text = preprocess(tweet)
  corpus.append(text)



In [ ]:
corpus



**Creating TF-IDF(Term Frequency and Inverse Document Frequency)**



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count=CountVectorizer().fit(corpus)


**Extracting all tokens i.e Features**

In [ ]:
count_tokens=count.get_feature_names()
count_tokens


**Transforming the corpus into Vector**

In [ ]:
article_vect = count.transform(corpus)
article_vect.toarray()

**Displaying the final Converted Vector with Features**

In [ ]:
df_count_vect=pd.DataFrame(data=article_vect.toarray(),columns=count_tokens)
df_count_vect

In [ ]:
def CountVec(df):
    corpus = []
    for  tweet in df.tweet :
      text = preprocess(tweet)
      corpus.append(text)

    count = CountVectorizer().fit(corpus)

    count_tokens = count.get_feature_names()
    article_vect = count.transform(corpus)

    df_count_vect = pd.DataFrame(data=article_vect.toarray(),columns=count_tokens)

    return df_count_vect

# Dimensionality Reduction based on Frequency Count


The goal is to reduce the dimension of the vectorized data by removing columns (features) based on their document frequency in other words if   the values of a specific column are not different from 0 in more than a specific threshold(seuil) we remove the column from the DateFrame 

**Vectorizing the data**

In [ ]:
dfVec = TfIdfVec(df)

In [ ]:
dfVec

**Selecting the list of columns that do not acheive a specific threshold(seuil)**

In [ ]:
subDf = dfVec[dfVec != 0]
subDf

In [ ]:
subDf.count(axis = 0)

In [ ]:
map = subDf.count(axis = 0) <= 5
map

**Droping the columns selected**

In [ ]:
dfVec.drop(dfVec.columns[map],axis=1)

In [ ]:
def redDim(dfVec,seuil):
    subDf = dfVec[dfVec != 0]
    map = subDf.count(axis = 0) <= seuil
    resultDf = dfVec.drop(dfVec.columns[map],axis=1)

    return resultDf